In [ ]:
import os
import struct
import pandas as pd
import numpy as np
import talib as tdx

In [ ]:
def readTdxLdayFile(fname="data/sh000001.day"):
  dataSet=[]
  with open(fname,'rb') as fl:
    buffer=fl.read() #读取数据到缓存
    size=len(buffer) 
    rowSize=32 #通信达day数据，每32个字节一组数据
    code=os.path.basename(fname).replace('.day','')
    for i in range(0,size,rowSize): #步长为32遍历buffer
      row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
      row[1]=row[1]/100
      row[2]=row[2]/100
      row[3]=row[3]/100
      row[4]=row[4]/100
      row.pop() #移除最后无意义字段
      row.insert(0,code)
      dataSet.append(row) 

  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
#   data=data.set_index(['tradeDate'])
  return code, data


In [ ]:
def select1(code, data):
    # 连续三日缩量
    cn = data.close.iloc[-1]
    ch= data.close.iloc[-1] * 1.1
    cl= data.close.iloc[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
#     df=pd.concat([tdx.MA(data.close, x) for x in (5,10,20,30,60,90,120,250,500,750,1000,1500,2000,2500,) ], axis = 1).dropna()[-1:]
    df=pd.concat([tdx.MA(data.close, x) for x in (5,10,20,30,60,90,120,250,500,750,1000,1500,2000,2500,) ], axis = 1)[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250', u'm500', u'm750', u'm1000', u'm1500', u'm2000', u'm2500']  
    df_c = df.m5 > df.m10
    df_h = df.apply(lambda x:cn > x ,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
    df['code'] =code
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)
    df=df.reset_index('tradeDate')
    df=df.set_index(['code','tradeDate'])
    return df


In [ ]:
def readPath(path):
  files = os.listdir(path)
  # codes=[]
  # dataSet=[]
  output=pd.DataFrame()
  for i in range(0,len(files)):
    fname = os.path.join(path,files[i])
    if os.path.isdir(fname):
      continue
    code, df = readTdxLdayFile(fname)
    cdf = select1(code, df)
    if cdf is not None:
      output=output.append(cdf)
  # data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
  # data=data.set_index(['code','tradeDate'])
  return output


In [ ]:
output=readPath('data') #读取目录下面的所有文件

In [ ]:
code, df = readTdxLdayFile('/home/zhangjx/backup/bk/easyquant/tdx/data/sh000001.day')

In [ ]:
df

In [ ]:
da=data_df.reset_index('tradeDate')

In [ ]:
import datetime

In [ ]:
pd.to_datetime(da.tradeDate)

In [ ]:
# data_df.to_csv('test.csv')

In [ ]:
data_df.index[,-1:-1]

In [ ]:
def select1(code,data):
    # 连续三日缩量
    ch= data.close.iloc[-1] * 1.1
    cl= data.close.iloc[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
    df=pd.concat([tdx.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1).dropna()[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250']  
    df_h = df.apply(lambda x:x.max() <= ch,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
    df['code'] =code
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)
    df=df.reset_index('tradeDate')
    df=df.set_index(['code','tradeDate'])
    return df


In [ ]:
bbb=select1('sh000001',data_df.loc['sh000001',])

In [ ]:
bbb

In [ ]:
bbb=bbb.set_index(['code','tradeDate'])

In [ ]:
data=bbb.set_index(['code','tradeDate'])

In [ ]:
output=None
for code in codes:
  aaa=data_df.loc[code,]
  out=select1(code, aaa)
  if output is None:
    output = out
  else:
#     print(code)
    output=output.append(out)

In [ ]:
output

In [ ]:
output.query('dfh==True and dfl==True').to_csv('out1.csv')

In [ ]:
bb=select1('000001',aaa)

In [ ]:
type(bb)

In [ ]:
import talib as tdx

In [ ]:
aaa=pd.read_csv('test.csv')

In [ ]:
aaa.set_index('vol').sort_index()

In [ ]:
df=readTdxLdayFile()
df['mon'] = df.tradeDate.apply(lambda x : str(x)[0:6])
df=df.set_index(['tradeDate'])

In [ ]:
dfmax=df.groupby(['mon']).apply(lambda x: x[x.high ==x.high.max()])
dfmax.drop_duplicates(subset=['high','mon'],keep='first',inplace=True)
dfmin=df.groupby(['mon']).apply(lambda x: x[x.low ==x.low.min()])
dfmin.drop_duplicates(subset=['low','mon'],keep='first',inplace=True)

In [ ]:
dfmax.to_csv('max.csv')

In [ ]:
dfmin.to_csv('min.csv')

In [ ]:
dfmax

In [ ]:
for x in dfmax.index:
  print(df.loc[x[1]])